<a href="https://colab.research.google.com/github/AndreCuenca17/Python_Quimica/blob/main/Electroquimica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np

symbols = [
    "La",
    "K",
    "Sr",
    "Ce",
    "Th",
    "Na",
    "Y",
    "Li",
    "Pr",
    "Pa",
    "Nd",
    "Pm",
    "Sm",
    "Gd",
    "Dy",
    "Zr",
    "Er",
    "Tm",
    "Yb",
    "Lu",
    "Ta",
    "Cm",
    "W",
    "U",
    "Bk",
    "Cf",
    "Es",
    "Fm",
    "Md",
    "No",
    "Lr",
    "Rf",
    "Db",
    "Mg",
    "Nb",
    "Ca",
    "Am",
    "Pu",
    "Re",
    "Np",
    "Sc",
    "Cr",
    "Be",
    "Mo",
    "Al",
    "Bi",
    "Ti",
    "Cu",
    "V",
    "In",
    "Sn",
    "Zn",
    "Mn",
    "Fe",
    "Si",
    "Ni",
    "Ru",
    "Ir",
    "Co",
    "Cd",
    "Sb",
    "Pb",
    "Rn",
    "Ga",
    "At",
    "B",
    "I",
    "Xe",
    "Tc",
    "As",
    "P",
    # Exception
    "Rh",
    "Ag",
    "Pt",
    "Au",
    "Fr",
    "Pd",
    "Hg",
    "Po",
    "Os",
    "Tl",
    "C",
    "Se",
    "S",
    "Ba",
    "Cs",
    "Kr",
    "N",
    "Cl",
    "O",
    "F",
    "He",
    "Ne",
    "Ar",
    "Rb",
    "Eu",
    "Tb",
    "Ho",
    "Hf",
    "Ra",
    "Sg",
    "Bh",
    "Hs",
    "Mt",
    "Ds",
    "Rg",
    "Uub",
    "Uut",
    "Uuq",
    "Uup",
    "Uuh",
    "Uus",
    "Uuo",
    "H"
]

class Molecule:
    def __init__(self, molecule: str, coff: int = 1):
        molecule_regex = '(.+?)\[(.+?)\]'
        molecule_pattern = re.compile(molecule_regex, re.UNICODE)
        (molecule, charge) = re.match(molecule_pattern, molecule).groups()

        atoms = {}
        atom_regex = '(\D{1,2})(\d)'
        atom_pattern = re.compile(atom_regex, re.UNICODE)
        central_atom_index = 118

        for atom in re.findall(atom_pattern, molecule):
            (symbol, count) = atom
            # print(symbol, count)
            atoms[symbol] = int(count)

            # UPDATE: Properly find central atom
            if central_atom_index > symbols.index(symbol):
                central_atom_index = symbols.index(symbol)

        self.atoms = atoms
        self.central_atom = symbols[central_atom_index]
        self.charge = int(charge)
        self.coff = int(coff)

    def get_atom(self, atom: str):
        return self.atoms[atom] * self.coff if atom in self.atoms else 0

    # UPDATE: Conversion from ionic to covalent
    def set_atom(self, atom: str, num: int):
        if num <= 0:
            del self.atoms[atom]
        else:
            self.atoms[atom] = num

    def get_central_atom(self):
        return self.atoms[self.central_atom] * self.coff

    def get_charge(self):
        return self.charge * self.coff

    # UPDATE: Properly find Oxidation State
    def get_os(self):
        total_charge = self.charge
        ligand_charge = (self.get_atom('H')*(+1) + self.get_atom('O')*(-2) + self.get_atom('S')*(-2))
        return (total_charge - ligand_charge) / self.get_central_atom()


class Reaction:
    def __init__(self, reactants: str = '', products: str = ''):
        def parse_side(reaction):
            molecules = {}
            molecule_regex = '\((.+?)\)(.+?\])'
            molecule_pattern = re.compile(molecule_regex)

            for molecule in re.findall(molecule_pattern, reaction):
                (coff, symbol) = molecule
                molecules[symbol] = Molecule(symbol, coff)

            return molecules

        self.reactants = parse_side(reactants)
        self.products = parse_side(products)

    def print_rxn(self, before: str = '', after: str = ''):
        def print_side(side):
            for symbol in side:
                molecule = side[symbol]
                print('(', molecule.coff, ')',  sep='', end='')

                for atom in molecule.atoms:
                    print(atom, sep='', end='')
                    print(molecule.atoms[atom], sep='', end='')

                print('[', molecule.charge, ']', ' + ',  sep='', end='')

        print(before, end='')
        print_side(self.reactants)
        print('\b\b\b', ' ---> ', end='')

        print_side(self.products)
        print('\b\b\b', '      ', end='')
        print(after, '\n')

    def get_side(self, side: str):
        side = self.reactants if side == 'reactant' else self.products
        return list(side.values())

    def add_molecule(self, symbol: str, coff: int = 1):
        # Positive Coff means in reactant side
        # Negative Coff means in product side
        same_side = self.reactants if coff >= 0 else self.products
        opp_side = self.reactants if coff < 0 else self.products
        coff = -abs(coff)
        # Adding to the opposite side with negative sign
        if symbol in opp_side:
            opp_side[symbol].coff += coff
        else:
            opp_side[symbol] = Molecule(symbol, coff)

        # Removing form the opposite side if needed
        if opp_side[symbol].coff > 0:
            return

        coff = -opp_side[symbol].coff
        opp_side.pop(symbol)

        # Adding to the same side form the opposite side sign inverted
        if symbol in same_side:
            same_side[symbol].coff += coff
        else:
            same_side[symbol] = Molecule(symbol, coff)

        # Removing form the same side if needed
        if same_side[symbol].coff > 0:
            return

        coff = 0
        same_side.pop(symbol)

    def remove_molecule(self, symbol: str, coff: int = 0):
        if coff > 0:
            if symbol in self.reactants:
                self.reactants[symbol].coff -= min(self.reactants[symbol].coff, abs(coff))
        elif coff < 0:
            if symbol in self.products:
                self.products[symbol].coff -= min(self.products[symbol].coff, abs(coff))
        else:
            if symbol in self.reactants:
                self.reactants.pop(symbol)

            if symbol in self.products:
                self.products.pop(symbol)

    def split_rxn(self):
        reduction_reaction = Reaction()
        oxidation_reaction = Reaction()

        for reactant_symbol in self.reactants:
            for product_symbol in self.products:
                reactant = self.reactants[reactant_symbol]
                product = self.products[product_symbol]

                if reactant.central_atom == product.central_atom:
                    if reactant.get_os() > product.get_os():
                        reduction_reaction.add_molecule(reactant_symbol, reactant.coff)
                        reduction_reaction.add_molecule(product_symbol, -product.coff)
                    else:
                        oxidation_reaction.add_molecule(reactant_symbol, reactant.coff)
                        oxidation_reaction.add_molecule(product_symbol, -product.coff)

        return (reduction_reaction, oxidation_reaction)

    def merge_rxn(self, reaction):
        def merge_side(side, side_rxn):
            side = 1 if side == 'reactant' else -1

            for molecule_symbol in side_rxn:
                molecule = side_rxn[molecule_symbol]
                self.add_molecule(molecule_symbol, side * molecule.coff)

        merge_side('reactant', reaction.reactants)
        merge_side('product', reaction.products)

        return self

def balance_medium(reaction: Reaction, medium: str):
    # Balancing Central Atom Start
    lcm = np.lcm(reaction.get_side('reactant')[0].get_central_atom(), reaction.get_side('product')[0].get_central_atom())
    multiplier = {'reactant': int(lcm / reaction.get_side('reactant')[0].get_central_atom()), 'product': int(lcm / reaction.get_side('product')[0].get_central_atom())}

    for molecule in reaction.reactants:
        reaction.reactants[molecule].coff *= multiplier['reactant']
    for molecule in reaction.products:
        reaction.products[molecule].coff *= multiplier['product']
    # Balancing Central Atom End
    # print(reactant, product)

    # Balancing Oxygen Atom Start
    water_added = sum(molecule.get_atom('O') for molecule in reaction.get_side('reactant')) - sum(elem.get_atom('O') for elem in reaction.get_side('product'))
    # print(water_added)
    reaction.add_molecule('H2O1[0]', -water_added)
    # Balancing Oxygen Atom End
    # print(reactant, product)

    if medium == 'acid':
        # Balancing Hydrogen Atom Start
        hydrogen_added = sum(molecule.get_atom('H') for molecule in reaction.get_side('reactant')) - sum(elem.get_atom('H') for elem in reaction.get_side('product'))
        # print(hydrogen_added)
        reaction.add_molecule('H1[+1]', -hydrogen_added)
        # Balancing Oxygen Atom End
        # print(reactant, product)
    else:
        # Balancing Hydrogen Atom Start
        water_added = sum(molecule.get_atom('H') for molecule in reaction.get_side('reactant')) - sum(elem.get_atom('H') for elem in reaction.get_side('product'))
        # print(water_added)
        reaction.add_molecule('H2O1[0]', -water_added)
        # Balancing Hydrogen Atom End
        # print(reactant, product)

        # Coverting to base medium
        hydroxide_added = water_added
        # print(hydroxide_added)
        reaction.add_molecule('O1H1[-1]', hydroxide_added)
        # print(reactant, product)

    return reaction


def balance_electron(red_rxn: Reaction, ox_rxn: Reaction):
    reduction_transferred_electron = abs(sum(molecule.get_charge() for molecule in red_rxn.get_side('reactant')) - sum(elem.get_charge() for elem in red_rxn.get_side('product')))
    oxidation_transferred_electron = abs(sum(molecule.get_charge() for molecule in ox_rxn.get_side('product')) - sum(elem.get_charge() for elem in ox_rxn.get_side('reactant')))

    # print(reduction_transferred_electron, oxidation_transferred_electron)
    total_transferred_electron = np.lcm(reduction_transferred_electron, oxidation_transferred_electron)
    # print(total_transferred_electron)
    multiplier = {'red': int(total_transferred_electron / reduction_transferred_electron), 'ox': int(total_transferred_electron / oxidation_transferred_electron)}
    # print(multiplier)

    for molecule in red_rxn.reactants:
        red_rxn.reactants[molecule].coff *= multiplier['red']

    for molecule in red_rxn.products:
        red_rxn.products[molecule].coff *= multiplier['red']

    for molecule in ox_rxn.reactants:
        ox_rxn.reactants[molecule].coff *= multiplier['ox']

    for molecule in ox_rxn.products:
        ox_rxn.products[molecule].coff *= multiplier['ox']

    # print(red_rxn)
    # print(ox_rxn)
    return (red_rxn, ox_rxn)

def main():
  #funciona con varios ejemplos
    # FeO2(s) + H2S(aq) --> Fe2+(aq) + SO42-(aq)
    reaction = Reaction('(1)Fe1O2[0] + (1)H2S1[0]', '(1)Fe1[+2] + (1)S1O4[-2]')

    # Zn + Cu2+ ---> Zn2+ + Cu
    # reaction = Reaction('(1)Zn1[0] + (1)Cu1[+2]', '(1)Zn1[+2] + (1)Cu1[0]')

    # As2S5(s) + NO3¯(aq) ---> H3AsO4(aq) + HSO4¯(aq) + NO2(g)
    # reaction = Reaction('(1)As2S5[0] + (1)N1O3[-1]', '(1)H3As1O4[0] + (1)H1S1O4[-1] + (1)N1O2[0]')

    # ClO3¯ + SO2 ---> SO42¯ + Cl¯
    # reaction = Reaction('(1)Cl1O3[-1] + (1)S1O2[0]', '(1)S1O4[-2] + (1)Cl1[-1]')

    # P4 ---> H2PO2 + PH3
    # reaction = Reaction('(1)P4[0]', '(1)H2P1O2[0] + (1)P1H3[0]')

    # Cl2 ---> ClO- + Cl-
    # reaction = Reaction('(1)Cl2[0]', '(1)Cl1O1[-1] + (1)Cl1[-1]')

    # H2O2 ---> H2O + O2
    # reaction = Reaction('(1)H2O2[0]', '(1)H2O1[0] + (1)O2[0]')

    # NH3 + ClO¯ ---> N2H4 + Cl¯
    # reaction = Reaction('(1)N1H3[0] + (1)Cl1O1[-1]', '(1)N2H4[0] + (1)Cl1[-1]')
    reaction.print_rxn('Initial: ')

    (red_rxn, ox_rxn) = reaction.split_rxn()
    red_rxn.print_rxn('Reduction: ')
    ox_rxn.print_rxn('Oxidation: ')

    red_rxn = balance_medium(red_rxn, 'acid')
    red_rxn.print_rxn('Reduction After Medium Balance: ')
    ox_rxn = balance_medium(ox_rxn, 'acid')
    ox_rxn.print_rxn('Oxidation After Medium Balance: ')

    (red_rxn, ox_rxn) = balance_electron(red_rxn, ox_rxn)
    red_rxn.print_rxn('Reduction After Balance Electron: ')
    ox_rxn.print_rxn('Oxidation After Balance Electron: ')

    reaction = red_rxn.merge_rxn(ox_rxn)
    reaction.print_rxn('Final: ')


if __name__ == "__main__":
    main()

Initial: (1)Fe1O2[0] + (1)H2S1[0] +   ---> (1)Fe1[2] + (1)S1O4[-2] +         

Reduction: (1)Fe1O2[0] +   ---> (1)Fe1[2] +         

Oxidation: (1)H2S1[0] +   ---> (1)S1O4[-2] +         

Reduction After Medium Balance: (1)Fe1O2[0] + (4)H1[1] +   ---> (1)Fe1[2] + (2)H2O1[0] +         

Oxidation After Medium Balance: (1)H2S1[0] + (4)H2O1[0] +   ---> (1)S1O4[-2] + (10)H1[1] +         

Reduction After Balance Electron: (4)Fe1O2[0] + (16)H1[1] +   ---> (4)Fe1[2] + (8)H2O1[0] +         

Oxidation After Balance Electron: (1)H2S1[0] + (4)H2O1[0] +   ---> (1)S1O4[-2] + (10)H1[1] +         

Final: (4)Fe1O2[0] + (6)H1[1] + (1)H2S1[0] +   ---> (4)Fe1[2] + (4)H2O1[0] + (1)S1O4[-2] +         

